In [2]:
import os
import sys
from typing import List, Dict, Any
from langchain_core.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic.v1 import BaseModel, Field
from openai import OpenAIError
from dotenv import load_dotenv
from tabulate import tabulate
from IPython.display import HTML, display
from openpyxl import Workbook
from openpyxl import reader
import pandas as pd

loaded = load_dotenv()

In [3]:
def get_BReqs_from_ws() -> pd.DataFrame:
    wb = reader.excel.load_workbook(filename="PoC-v1-Inputs.xlsx", read_only=True, keep_vba=False, data_only=False, keep_links=True, rich_text=False)
    ws_br = wb["BReqs"]
    df = pd.DataFrame(ws_br.values)
    wb.close()
    df = df.rename(columns={0: "Name", 1: "Description"})
    #print(df.head())
    return df

def get_BReq_context():
    df = get_BReqs_from_ws()
    result = ""
    for index, row in df.iterrows():
        result += "Requirement-" + str(row["Name"]) + " is that " + str(row["Description"]) + ". "
    return result

In [88]:
class Story(BaseModel):
    title: str = Field(description="The title of the story")
    story_text: str = Field(description="The user story in the 'As a... I want to... So That...' format.")
    relevance_score: float = Field(description="The relevance of the story to the scenario. 0 is the worst, 10 is the best.")

class Stories(BaseModel):
    contents: List[Story] = Field(description="A list of Stories")

def create_chat_model(temperature: float = 0.0) -> ChatOpenAI:
    try:
        return ChatOpenAI(temperature=temperature)
    except OpenAIError as e:
        print(f"Error creating ChatOpenAI model: {e}")
        sys.exit(1)

def create_parser() -> PydanticOutputParser:
    return PydanticOutputParser(pydantic_object=Stories)

def create_chat_prompt(template: str) -> ChatPromptTemplate:
    system_message_prompt = SystemMessagePromptTemplate.from_template(template)
    return ChatPromptTemplate.from_messages([system_message_prompt])

def generate_stories(model: ChatOpenAI, chat_prompt: ChatPromptTemplate, parser: PydanticOutputParser,
                   input_data: Dict[str, Any]) -> Stories:
    try:
        prompt_and_model = chat_prompt | model
        result = prompt_and_model.invoke(input_data)
        return parser.parse(result.content)
    except OpenAIError as e:
        print(f"Error generating names: {e}")
        return Stories(contents=[])
    except ValueError as e:
        print(f"Error parsing output: {e}")
        return Stories(contents=[])

def get_stories(feature: str):
    if 'OPENAI_API_KEY' not in os.environ:
        print("Error: OPENAI_API_KEY environment variable is not set.")
        sys.exit(1)

    breqs = get_BReq_context()
    print(breqs)
    
    principles = """
    - The story should evoke the core goals of users of the {feature} feature.
    - The title should be between 3 and 8 words long.
    - The user story in the 'As a… I want to… So That…' format. 
    - The stories should not break the following business requirements {breqs}.
    """

    template = """Generate the 5 most relevent user stories for a new custom software capability that offers a {feature} feature.
    You must follow the following principles: {principles}
    {format_instructions}
    """

    result = Stories(contents=[])
    try:
        model = create_chat_model()
        parser = create_parser()
        chat_prompt = create_chat_prompt(template)

        input_data = {
            "breqs": breqs,
            "principles": principles,
            "feature": feature,
            "format_instructions": parser.get_format_instructions(),
        }
        #print(input_data)

        # Method 1: Using generate_stories function
        result = generate_stories(model, chat_prompt, parser, input_data)
        #print("Method 1 result:")
        return result.contents
        
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        sys.exit(1)

stories = get_stories("An online shopping cart")
print(stories)

Requirement-Order-Timestamp is that All placed orders must be timestamped as 'order received' at a time between when the user clicked the 'Place order' button and when the payment processor authorized the payment.. 
[Story(title='Add items to cart', story_text='As a shopper, I want to easily add items to my online shopping cart, so that I can review and purchase them later.', relevance_score=9.0), Story(title='View cart total', story_text='As a customer, I want to see the total cost of items in my shopping cart, so that I can budget and plan my purchase.', relevance_score=8.0), Story(title='Apply promo codes', story_text='As a user, I want to be able to apply promo codes to my shopping cart, so that I can avail discounts on my purchases.', relevance_score=7.0), Story(title='Save cart for later', story_text='As a shopper, I want to save my shopping cart for later, so that I can come back and complete my purchase at a convenient time.', relevance_score=8.0), Story(title='Remove items fro

In [89]:
stories_dicts = [story.__dict__ for story in stories]
table = tabulate(stories_dicts, tablefmt='html', headers="keys")
display(HTML(table))

title,story_text,relevance_score
Add items to cart,"As a shopper, I want to easily add items to my online shopping cart, so that I can review and purchase them later.",9
View cart total,"As a customer, I want to see the total cost of items in my shopping cart, so that I can budget and plan my purchase.",8
Apply promo codes,"As a user, I want to be able to apply promo codes to my shopping cart, so that I can avail discounts on my purchases.",7
Save cart for later,"As a shopper, I want to save my shopping cart for later, so that I can come back and complete my purchase at a convenient time.",8
Remove items from cart,"As a buyer, I want to easily remove items from my shopping cart, so that I can adjust my purchase before checkout.",9
